# 5장 - Support Vector Machine
## 선형 SVM 분류
SVM 분류기는 클래스 사이에 가장 폭이 넓은 도로를 찾는 것으로 생각 할 수 있어서 **라지 마진 분류**라고 한다. 각 경계에 위치한 샘플을 **서포트 벡터** 라고 한다. SVM도 스케일 통일이 중요하다

## 소프트 마진 분류
모든 샘플이 도로 바깥쪽에 분류되어 있으면 **Hard Margin Classification** 이라고 부른다. 하드 마진 분류의 문제점은 데이터가 선형적으로 구분될 수 있어야 잘 동작하고 이상치에 민감하다. 그래서 유연하게 도로의 폭을 넓히거나 **Margin Violation** (샘플이 도로 중간이나 반대에 있는 경우, 역주행?) 을 잡아야 하는데 이런 것을 **Soft Margin Classification** 이라고 한다.

사이킷런에서는 C 하이퍼파라미터로 균형을 조절하는데 C 값과 마진 오류는 반비례한다.

In [2]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris['data'][:, (2, 3)] # 꽃잎 길이, 꽃잎 너비
y = (iris['target'] == 2).astype(np.float64)

svm_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('linear_svc', LinearSVC(C=1, loss='hinge'))
])

svm_clf.fit(X, y)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('linear_svc', LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0))])

In [3]:
svm_clf.predict([[5.5, 1.7]])

array([1.])

## 비선형 SVM 분류

비선형 데이터셋은 4장처럼 다항 특성과 같은 특성을 추가하는 것이다. 하지만 낮은 차슈는 복잡한 데이터셋을 잘 표현하지 못하고 높은 차수는 모델이 느려진다. 다행히 SVM은 커널 트릭이라는 실제로 특성을 추가하지 않으면서 다항식 특성을 많이 추가한 것 같은 효과를 누릴 수 있다.

이떄 모델이 오버피팅되면 차수를 줄여야하고 과소적합이면 차수를 늘려야 한다. coef0이 모델이 높은 차수와 낮은 차수에 영향을 얼마나 받을지 조절한다. 적절한 하이퍼파라미터를 찾으려면 그리드 탐색을 하는 것이 좋다.

샘플이 특정 랜드마크와 얼마나 닮았는지 추정하는 유사도 함수로 계산한 특성을 추가하는 것도 좋다. 

In [6]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

X, y = make_moons(n_samples=100, noise=0.15, random_state=42)

polynomial_svm_clf = Pipeline([
    ('poly_features', PolynomialFeatures(degree=3)),
    ('scaler', StandardScaler()),
    ('svm_clf', LinearSVC(C=10, loss='hinge'))
])

polynomial_svm_clf.fit(X, y)

Pipeline(memory=None,
     steps=[('poly_features', PolynomialFeatures(degree=3, include_bias=True, interaction_only=False)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svm_clf', LinearSVC(C=10, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0))])

In [7]:
from sklearn.svm import SVC
poly_kernel_svm_clf = Pipeline([
    ('scaler', StandardScaler()),
    ('svm_clf', SVC(kernel='poly', degree=3, coef0=1, C=5))
])

poly_kernel_svm_clf.fit(X, y)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svm_clf', SVC(C=5, cache_size=200, class_weight=None, coef0=1,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])

## 5.5 연습문제

1. SVM의 근본 아이디어는?

구분해야 하는 각 클래스 사이에 가장 폭이 넓은 도로를 찾는 알고리즘이라고 할 수 있다. 여기서 도로라는 것은 도로의 경계는 각 클래스의 경계값을 이용해서 직선을 그은 것이고 중앙선은 각 경계로부터 최대한 멀리 떨어져야 샘플들이 혼동 없이 구분될 것이다

2. 서포트 벡터란?

서포트 벡터는 위에서 말한 것처럼 선을 긋기 위해 사용되는 클래스 별 샘플들의 경계값을 말한다.

3. SVM의 입력값의 스케일이 중요한 이유는?

어느 한 축의 스케일이 커버리면 도로의 기울기가 무척 작아지기 때문에 결정 경계가 좋아지지 않는다.

4. SVM이 샘플을 분류할떄 신뢰도 점수나 확률을 출력할 수 있는지?

책만 보면 그렇지 않은 것 같다.

8. MNIST 와 SVM

In [2]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC

In [58]:
ova_clf = OneVsRestClassifier(LinearSVC(C=100, loss='hinge'))

In [4]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')

In [48]:
X, y = mnist['data'], mnist['target']
X.shape # 784 = 28*28

(70000, 784)

In [65]:
size = 60000
X_train, X_test, y_train, y_test = X[:size], X[size:], y[:size], y[size:]

In [66]:
import numpy as np

shuffle_index = np.random.permutation(size)
X_train, y_train = X_train[shuffle_index], y_train[shuffle_index]

In [38]:
train_size = 5000

# X_train, y_train = X_train[:size], y_train[:size]

OneVsRestClassifier(estimator=LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0),
          n_jobs=1)

In [67]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train.astype(np.float32))

In [68]:
ova_clf.fit(X_train, y_train)

OneVsRestClassifier(estimator=LinearSVC(C=100, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='hinge', max_iter=1000, multi_class='ovr',
     penalty='l2', random_state=None, tol=0.0001, verbose=0),
          n_jobs=1)

In [69]:
from sklearn.model_selection import cross_val_score

cross_val_score(ova_clf, X_train, y_train, cv=5)

array([0.86214077, 0.86935511, 0.86408333, 0.87063433, 0.87620874])